#### 今回使用するマシンスペック

In [3]:
import platform
import multiprocessing as mp
import timeit

def print_sysinfo():
    print( '\nPython version  :', platform.python_version() )
    print( 'compiler        :', platform.python_compiler() )
    
    print( '\nsystem     :', platform.system() )
    print( 'release    :', platform.release() )
    print( 'machine    :', platform.machine() )
    print( 'processor  :', platform.processor() )
    print( 'CPU count  :', mp.cpu_count() )
    print( 'interpreter:', platform.architecture()[0] )
    print

In [4]:
print_sysinfo()


Python version  : 3.6.1
compiler        : GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)

system     : Darwin
release    : 16.7.0
machine    : x86_64
processor  : i386
CPU count  : 4
interpreter: 64bit


### Daskを使った簡単な並列処理

以下に簡単な計算例を示す。

In [5]:
from time import sleep

def slowadd(x,y):
    """ addをスローで行う """
    sleep(1)
    return x+y

def slowinc(x):
    """ incrementをスローで行う """
    sleep(1)
    return x+1

def slowsum(L):
    """ sumをスローで行う """
    sleep(1)
    return sum(L)

実行後は以下のようになる。スリープで遅延させているため処理時間は１sとなる。

In [6]:
%time slowsum([1,2,3])

CPU times: user 842 µs, sys: 840 µs, total: 1.68 ms
Wall time: 1 s


6

上記で宣言した関数を用いて簡単な処理を書く。  
リストを３つ作成し中身を足して行くだけの簡単な処理である。  
実行には12sほどかかっている。

In [7]:
%%time

data = [1,2,3]
A = [slowinc(i) for i in data]
B = [slowadd(a,10) for a in A]
C = [slowadd(b,100) for b in B]
score = slowsum(A) + slowsum(B) + slowsum(C)

print(score)

387
CPU times: user 1.11 ms, sys: 1.2 ms, total: 2.31 ms
Wall time: 12 s


### Daskを使って並列化

In [8]:
from dask import delayed

In [9]:
%%time

data = [1,2,3]
A = [delayed(slowinc)(i) for i in data]
B = [delayed(slowadd)(b,10) for b in A]
C = [delayed(slowadd)(c,100) for c in B]
score = delayed(slowsum)(A) + delayed(slowsum)(B) + delayed(slowsum)(C)

print(score)

Delayed('add-32e6d588992ef95a872195a6959d5925')
CPU times: user 2.3 ms, sys: 1.69 ms, total: 3.99 ms
Wall time: 3.89 ms


処理時間は短くなっているがここでは計算は行なっておらず、グラフのマッピングを行なっている。  
以下にグラフのマッピング結果を示す。

In [10]:
score.visualize()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/art/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-97007b5102e0>", line 1, in <module>
    score.visualize()
  File "/Users/art/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/dask/base.py", line 69, in visualize
    optimize_graph=optimize_graph, **kwargs)
  File "/Users/art/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/dask/base.py", line 265, in visualize
    return dot_graph(dsk, filename=filename, **kwargs)
  File "/Users/art/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/dask/dot.py", line 246, in dot_graph
    with open(full_filename, 'wb') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'mydask.png'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/art/.pyenv/

FileNotFoundError: [Errno 2] No such file or directory: 'mydask.png'

In [ ]:
%%time
score.compute()

daskでは並列処理のマッピングを自動で行ってくれる

### bokehを使って処理段階を描写

In [ ]:
from bokeh.plotting import output_notebook
output_notebook()

In [ ]:
from dask.diagnostics import Profiler
prof = Profiler()

with prof:
    score.compute()

prof.visualize()

### 辞書型で並列処理を行う

In [ ]:
# 並列処理に使用する関数を定義しておく
def inc(x):
    return x+1

def add(x,y):
    return x+y

In [ ]:
# daskでは辞書型を用いてグラフを表現する
dask_dict = {
    "a":1,
    "b":(inc,"a"),
    "x":10,
    "y":(inc,"x"),
    "z":(add,"b","y")
}

In [ ]:
dask_dict

In [ ]:
from dask.multiprocessing import get
get(dask_dict,"z")

In [ ]:
from dask.dot import dot_graph
dot_graph(dask_dict)

daskでは辞書による宣言を予め行なっておけば並列化処理を自動で行なってくれる。

### モンテカルロ法で速度比較を行う

In [ ]:
import random

In [ ]:
def is_incide_circle():
    """ランダムな1点を作成し、指定した円の範囲内にあるかどうかを判定する"""
    x = random.random()
    y = random.random()
    if x**2 + y**2 <= 1:
        return 1
    else:
        return 0

In [ ]:
def compute_pi(num):
    """引数回シュミレーションを行い、円周率の計算を行う"""
    count = [is_incide_circle() for i in range(num)]
    return 4*sum(count)/num

In [ ]:
%%time
print(compute_pi(100))

並列実装なしで点を1000万個プロットした場合、処理時間は約6sとなっている。

#### モンテカルロを並列実装

In [ ]:
def parallel_compute1_pi(num):
    """引数回シュミレーションを並列実行で行い、円周率の計算を行う"""
    count = [delayed(is_incide_circle)() for i in range(num)]
    return 4*delayed(sum)(count)/num

In [ ]:
%%time
print(parallel_compute1_pi(100).compute())

処理が遅くなっている。処理の流れを見てみる

In [ ]:
score = parallel_compute1_pi(10)
score.visualize()

is_incide_circleメソッドが処理の回数分呼ばれている。  
メソッドの内部処理は非常に小さなタスクのため複数回呼び出すごとに処理時間のオーバーヘッドが生じてしまい処理が遅くなっていると思われる。  
ここでis_incide_cicleを呼び出す関数を新たに作成することで処理時間のオーバーヘッドを解消する。  

In [ ]:
def how_many_inside_circle(k):
    return sum(is_incide_circle() for i in range(int(k)))

In [ ]:
def parallel_compute2_pi(nsamples, k):
    points = [delayed(how_many_inside_circle)(k) for i in range(int(nsamples / k))]
    if nsamples % k != 0: 
        points.append(delayed(how_many_inside_circle)(nsamples % k))
    return 4. * delayed(sum)(points) / nsamples

In [ ]:
%%time
print(parallel_compute_pi(100,10).compute())

In [ ]:
score = parallel_compute_pi(15,3)

In [ ]:
score.visualize()

In [ ]:
def input():
    return "12/31 25:01"

In [ ]:
# 入力内容を保持
t = input()

def Correction_time(t):
    """時間を修正する関数"""
    
    # 標準入力から時刻を細分化
    month = t.split(" ")[0].split("/")[0]
    day = int(t.split(" ")[0].split("/")[1])
    hour = int(t.split(" ")[1].split(":")[0])
    time = t.split(" ")[1].split(":")[1]
    
    if 0<= hour <= 23: # 時間の値が標準の時刻内の場合
        pass
    else: # 標準の時刻を外れていた場合
        day = str(day + (int(hour/24)))
        hour = hour % 24
        if hour/10 <= 1:
            hour = "0" + str(hour)
        else:
            hour = str(hour)    
        t = month + "/" + day + " " + hour + ":" + time
    return t

In [ ]:
print(Correction_time(t))

In [ ]:
N = 3
hands_list = ["paper","paper","rock"]
hands = [i for i in set(hands_list)]

hands_dict = {
    "scissorspaper":"scissors",
    "paperscissors":"scissors",
    "rockscissors":"rock",
    "scissorsrock":"rock",
    "paperrock":"paper",
    "rockpaper":"paper",
}

if len(hands) == 3:
    print("draw")
elif len(hands) == 1:
    print("draw")
else:
    print(hands_dict[str(hands[0]) + str(hands[1])])